In [10]:
import gc
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn import model_selection

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
import tensorflow as tf
from numpy import genfromtxt
import datatable as dt
import pandas as pd

In [3]:
inputDate='2019-01-03 01:00:00'

In [4]:
data=np.load(f'/hdd/pred_prec_temp/{inputDate}.npy')

In [5]:
data[24:].shape

(1, 1159, 1505, 3)

In [6]:
# DateRange=pd.to_timedelta(1,unit='day')
# generateDate=[str(i) for i in pd.date_range(start=pd.to_datetime(inputDate)-DateRange,end=inputDate , freq='H')]

# for date_ in generateDate:
#     print(date_)
#     data_save(date_)

In [11]:
import gc
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn import model_selection

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl

In [12]:
import os


In [19]:
tf.convert_to_tensor(np.ndarray([1]),dtype=tf.float32)

TypeError: __init__() missing 1 required positional argument: 'value_index'

In [20]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, paths, test=False):
        self.paths = paths
        self.test = test

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        data = np.load(path)
        #torch.Size([24, 1159, 1505, 3])
        x = data[:24,:300,:300,:]
        #         x = x / 255.0
        x = x.astype(np.float32)
        x = torch.tensor(x, dtype=torch.float32)
        if self.test:
            return x
        else:
            y = data[24:,:300,:300,:]
            #             y = y / 255.0
            y = y.astype(np.float32)
            y = torch.tensor(y, dtype=torch.float32)
            return x, y

In [21]:
sorted(os.listdir('/hdd/pred_prec_temp'))[:-24]

['2019-01-01 00:00:00.npy',
 '2019-01-01 01:00:00.npy',
 '2019-01-01 02:00:00.npy',
 '2019-01-01 03:00:00.npy',
 '2019-01-01 04:00:00.npy',
 '2019-01-01 05:00:00.npy',
 '2019-01-01 06:00:00.npy',
 '2019-01-01 07:00:00.npy',
 '2019-01-01 08:00:00.npy',
 '2019-01-01 09:00:00.npy',
 '2019-01-01 10:00:00.npy',
 '2019-01-01 11:00:00.npy',
 '2019-01-01 12:00:00.npy',
 '2019-01-01 13:00:00.npy',
 '2019-01-01 14:00:00.npy',
 '2019-01-01 15:00:00.npy',
 '2019-01-01 16:00:00.npy',
 '2019-01-01 17:00:00.npy',
 '2019-01-01 18:00:00.npy',
 '2019-01-01 19:00:00.npy',
 '2019-01-01 20:00:00.npy',
 '2019-01-01 21:00:00.npy',
 '2019-01-01 22:00:00.npy',
 '2019-01-01 23:00:00.npy',
 '2019-01-02 00:00:00.npy',
 '2019-01-03 00:00:00.npy',
 '2019-01-03 01:00:00.npy',
 '2019-01-03 02:00:00.npy',
 '2019-01-03 03:00:00.npy',
 '2019-01-03 04:00:00.npy',
 '2019-01-03 05:00:00.npy',
 '2019-01-03 06:00:00.npy',
 '2019-01-03 07:00:00.npy',
 '2019-01-03 08:00:00.npy',
 '2019-01-03 09:00:00.npy',
 '2019-01-03 10:00:0

In [22]:
class DataModule(pl.LightningDataModule):
    def __init__(self, batch_size, test=False, num_workers=4):
        super().__init__()
        self.test = test
        self.batch_size = batch_size
        self.num_workers = 4

    def setup(self, stage="train"):
        if stage == "train":
            paths = [f'/hdd/pred_prec_temp/{i}' for i in sorted(os.listdir('/hdd/pred_prec_temp'))[:-24]]
            train_paths, val_paths = model_selection.train_test_split(
                paths, test_size=0.1, shuffle=True
            )
            self.train_dataset = Dataset(train_paths)
            self.val_dataset = Dataset(val_paths)
        else:
            test_paths = [f'/hdd/pred_prec_temp/{i}' for i in sorted(os.listdir('/hdd/pred_prec_temp'))[-24:]]
            self.test_dataset = Dataset(test_paths, test=True)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            pin_memory=True,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_dataset,
            batch_size=2 * self.batch_size,
            pin_memory=True,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_dataset,
            batch_size=2 * self.batch_size,
            pin_memory=True,
            num_workers=self.num_workers,
        )

In [23]:
# def visualize(x, y=None, test=False):
#     cmap = plt.cm.get_cmap("RdBu")
#     cmap = cmap.reversed()
#     if test:
#         fig, axes = plt.subplots(1, 4, figsize=(10, 10))
#         for i, ax in enumerate(axes):
#             img = x[:, :, i]
#             ax.imshow(img, cmap=cmap)
#     else:
#         fig, axes = plt.subplots(1, 5, figsize=(10, 10))
#         for i, ax in enumerate(axes[:-1]):
#             img = x[:, :, i]
#             ax.imshow(img, cmap=cmap)
#         axes[-1].imshow(y[:, :, 0], cmap=cmap)
#     #     plt.tight_layout()
#     plt.show()

In [24]:
datamodule = DataModule(batch_size=1)
datamodule.setup()

In [25]:
for batch in datamodule.train_dataloader():
    break

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/root/anaconda3/envs/jupyter/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/root/anaconda3/envs/jupyter/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/root/anaconda3/envs/jupyter/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py", line 83, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/root/anaconda3/envs/jupyter/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py", line 83, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/root/anaconda3/envs/jupyter/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py", line 85, in default_collate
    raise TypeError(default_collate_err_msg_format.format(elem_type))
Typ

In [ ]:
xs, ys = batch
x, y = xs[0], ys[0]

In [30]:
x.shape

torch.Size([24, 300, 300, 3])

In [31]:
y.shape

torch.Size([24, 300, 300, 3])

In [32]:
# x = x.permute(1, 2, 0).numpy()
# y = y.permute(1, 2, 0).numpy()
# visualize(x, y)
# break

In [ ]:
 data = np.load(path)
        #torch.Size([24, 1159, 1505, 3])
        x = data[:24,:300,:300,:]
        #         x = x / 255.0
        x = x.astype(np.float32)
        x = tf.convert_to_tensor(x,dtype=tf.float32)
        if self.test:
            return x
        else:
            y = data[1:,:300,:300,:]
            #             y = y / 255.0
            y = y.astype(np.float32)
            
            y = tf.convert_to_tensor(y,dtype=tf.float32)

In [13]:
paths = [f'/hdd/pred_prec_temp/{i}' for i in sorted(os.listdir('/hdd/pred_prec_temp'))[:-24]]
train_paths, val_paths = model_selection.train_test_split(
            paths, test_size=0.1, shuffle=False
        )

test_paths = [f'/hdd/pred_prec_temp/{i}' for i in sorted(os.listdir('/hdd/pred_prec_temp'))[-24:]]

In [14]:
train_x=list();train_y=list()
for path in train_paths:
    print(path)
    data=np.load(path)
    x=data[np.newaxis,:24,:300,:300]
    x=x.astype(np.float32)
    y=data[np.newaxis,1:,:300,:300]
    y=y.astype(np.float32)
    train_x.append(x)
    train_y.append(y)

/hdd/pred_prec_temp/2019-01-01 00:00:00.npy
/hdd/pred_prec_temp/2019-01-01 01:00:00.npy
/hdd/pred_prec_temp/2019-01-01 02:00:00.npy
/hdd/pred_prec_temp/2019-01-01 03:00:00.npy
/hdd/pred_prec_temp/2019-01-01 04:00:00.npy
/hdd/pred_prec_temp/2019-01-01 05:00:00.npy
/hdd/pred_prec_temp/2019-01-01 06:00:00.npy
/hdd/pred_prec_temp/2019-01-01 07:00:00.npy
/hdd/pred_prec_temp/2019-01-01 08:00:00.npy
/hdd/pred_prec_temp/2019-01-01 09:00:00.npy
/hdd/pred_prec_temp/2019-01-01 10:00:00.npy
/hdd/pred_prec_temp/2019-01-01 11:00:00.npy
/hdd/pred_prec_temp/2019-01-01 12:00:00.npy
/hdd/pred_prec_temp/2019-01-01 13:00:00.npy
/hdd/pred_prec_temp/2019-01-01 14:00:00.npy
/hdd/pred_prec_temp/2019-01-01 15:00:00.npy
/hdd/pred_prec_temp/2019-01-01 16:00:00.npy
/hdd/pred_prec_temp/2019-01-01 17:00:00.npy
/hdd/pred_prec_temp/2019-01-01 18:00:00.npy
/hdd/pred_prec_temp/2019-01-01 19:00:00.npy
/hdd/pred_prec_temp/2019-01-01 20:00:00.npy
/hdd/pred_prec_temp/2019-01-01 21:00:00.npy
/hdd/pred_prec_temp/2019-01-01 2

In [15]:
val_x=list();val_y=list()
for path in val_paths:
    print(path)
    data=np.load(path)
    x=data[np.newaxis,:24,:300,:300]
    x=x.astype(np.float32)
    y=data[np.newaxis,1:,:300,:300]
    y=y.astype(np.float32)
    val_x.append(x)
    val_y.append(y)

/hdd/pred_prec_temp/2019-01-04 16:00:00.npy
/hdd/pred_prec_temp/2019-01-04 17:00:00.npy
/hdd/pred_prec_temp/2019-01-04 18:00:00.npy
/hdd/pred_prec_temp/2019-01-04 19:00:00.npy
/hdd/pred_prec_temp/2019-01-04 20:00:00.npy
/hdd/pred_prec_temp/2019-01-04 21:00:00.npy
/hdd/pred_prec_temp/2019-01-04 22:00:00.npy
/hdd/pred_prec_temp/2019-01-04 23:00:00.npy


In [16]:
test_x=list();test_y=list()
for path in test_paths:
    print(path)
    data=np.load(path)
    x=data[np.newaxis,:24,:300,:300]
    x=x.astype(np.float32)
    y=data[np.newaxis,1:,:300,:300]
    y=y.astype(np.float32)
    test_x.append(x)
    test_y.append(y)

/hdd/pred_prec_temp/2019-01-05 00:00:00.npy
/hdd/pred_prec_temp/2019-01-05 01:00:00.npy
/hdd/pred_prec_temp/2019-01-05 02:00:00.npy
/hdd/pred_prec_temp/2019-01-05 03:00:00.npy
/hdd/pred_prec_temp/2019-01-05 04:00:00.npy
/hdd/pred_prec_temp/2019-01-05 05:00:00.npy
/hdd/pred_prec_temp/2019-01-05 06:00:00.npy
/hdd/pred_prec_temp/2019-01-05 07:00:00.npy
/hdd/pred_prec_temp/2019-01-05 08:00:00.npy
/hdd/pred_prec_temp/2019-01-05 09:00:00.npy
/hdd/pred_prec_temp/2019-01-05 10:00:00.npy
/hdd/pred_prec_temp/2019-01-05 11:00:00.npy
/hdd/pred_prec_temp/2019-01-05 12:00:00.npy
/hdd/pred_prec_temp/2019-01-05 13:00:00.npy
/hdd/pred_prec_temp/2019-01-05 14:00:00.npy
/hdd/pred_prec_temp/2019-01-05 15:00:00.npy
/hdd/pred_prec_temp/2019-01-05 16:00:00.npy
/hdd/pred_prec_temp/2019-01-05 17:00:00.npy
/hdd/pred_prec_temp/2019-01-05 18:00:00.npy
/hdd/pred_prec_temp/2019-01-05 19:00:00.npy
/hdd/pred_prec_temp/2019-01-05 20:00:00.npy
/hdd/pred_prec_temp/2019-01-05 21:00:00.npy
/hdd/pred_prec_temp/2019-01-05 2

In [18]:
Xtrain=np.concatenate(train_x)
Ytrain=np.concatenate(train_y)
Xval  =np.concatenate(val_x  )
Yval  =np.concatenate(val_y  )
Xtest =np.concatenate(test_x )
Ytest =np.concatenate(test_y )

In [76]:
Xtrain.shape

(65, 24, 300, 300, 3)

In [77]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pylab as plt

In [6]:
seq = keras.Sequential(
    [
        keras.Input(
            shape=(None, 300, 300, 3)
        ),  # Variable-length sequence of 40x40x1 frames
        layers.ConvLSTM2D(
            filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=1, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
#         layers.BatchNormalization(),
#         layers.ConvLSTM2D(
#             filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
#         ),
#         layers.BatchNormalization(),
#         layers.ConvLSTM2D(
#             filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
#         ),
#         layers.BatchNormalization(),
#         layers.Conv3D(
#             filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
#         ),
    ]
)
seq.compile(loss="binary_crossentropy", optimizer="adadelta")

In [35]:
x.shape

torch.Size([24, 300, 300, 3])

In [36]:
y.shape

torch.Size([24, 300, 300, 3])

In [84]:
Xtrain.dtype

dtype('float32')

In [19]:
np.save('/hdd/Xrain.npy',Xtrain)
np.save('/hdd/Ytrain.npy',Ytrain)
np.save('/hdd/Xtest.npy',Xtest)
np.save('/hdd/Ytest.npy',Ytest)
np.save('/hdd/Xval.npy',Xval)
np.save('/hdd/Yval.npy',Yval)

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pylab as plt

In [3]:
Xtrain=np.load('/hdd/Xrain.npy')
Ytrain=np.load('/hdd/Ytrain.npy')
Xtest=np.load('/hdd/Xtest.npy')
Ytest=np.load('/hdd/Ytest.npy')
Xval=np.load('/hdd/Xval.npy')
Yval=np.load('/hdd/Yval.npy')

In [4]:
Xtrain=tf.convert_to_tensor(Xtrain,dtype=tf.float32)
Ytrain=tf.convert_to_tensor(Ytrain,dtype=tf.float32)
Xtest=tf.convert_to_tensor(Xtest,dtype=tf.float32)
Ytest=tf.convert_to_tensor(Ytest,dtype=tf.float32)
Xval=tf.convert_to_tensor(Xval,dtype=tf.float32)
Yval=tf.convert_to_tensor(Yval,dtype=tf.float32)

################ 

# https://www.tensorflow.org/addons/tutorials/layers_normalizations

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pylab as plt

In [2]:
import tensorflow_addons as tfa

In [3]:
Xtrain=np.load('/hdd/Xrain.npy')
Ytrain=np.load('/hdd/Ytrain.npy')
Xtrain=Xtrain[:,:12]
Ytrain=Ytrain[:,:12]
Xtrain=tf.convert_to_tensor(Xtrain,dtype=tf.float32)
Ytrain=tf.convert_to_tensor(Ytrain,dtype=tf.float32)

In [4]:
Xval=np.load('/hdd/Xval.npy')
Yval=np.load('/hdd/Yval.npy')
Xval=Xval[:,:12]
Yval=Yval[:,:12]
Xval=tf.convert_to_tensor(Xval,dtype=tf.float32)
Yval=tf.convert_to_tensor(Yval,dtype=tf.float32)

In [5]:
Xtest=np.load('/hdd/Xtest.npy')
Ytest=np.load('/hdd/Ytest.npy')
Xtest=Xtest[:,:12]
Ytest=Ytest[:,:12]
Xtest=tf.convert_to_tensor(Xtest,dtype=tf.float32)
Ytest=tf.convert_to_tensor(Ytest,dtype=tf.float32)

In [6]:
Xtrain.shape

TensorShape([65, 12, 300, 300, 3])

In [7]:
seq = keras.Sequential(
    [
        keras.Input(
            shape=(None, 300, 300, 3)
        ),  # Variable-length sequence of 40x40x1 frames
        layers.ConvLSTM2D(
            filters=5, kernel_size=(3, 3), dropout=0.2, recurrent_dropout=0.1,
            padding="same", return_sequences=True
        ),
        #tfa.layers.GroupNormalization(groups=5, axis=4),
        tf.keras.layers.LayerNormalization(axis=4 , center=True , scale=True),

#         tfa.layers.InstanceNormalization(axis=4, 
#                                    center=True, 
#                                    scale=True,
#                                    beta_initializer="random_uniform",
#                                    gamma_initializer="random_uniform"),
#         layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=3, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
#         layers.BatchNormalization(),
#         layers.ConvLSTM2D(
#             filters=5, kernel_size=(3, 3), padding="same", return_sequences=True
#         ),
#         layers.BatchNormalization(),
#         layers.ConvLSTM2D(
#             filters=5, kernel_size=(3, 3), padding="same", return_sequences=True
#         ),
#         layers.BatchNormalization(),
#         layers.Conv3D(
#             filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
#         ),
    ]
)
seq.compile(loss="mae", optimizer=keras.optimizers.Adam(lr=0.01),metrics=['mse'])

In [8]:
seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, None, 300, 300, 5) 1460      
_________________________________________________________________
layer_normalization (LayerNo (None, None, 300, 300, 5) 10        
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 300, 300, 3) 876       
Total params: 2,346
Trainable params: 2,346
Non-trainable params: 0
_________________________________________________________________


In [9]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
seq.fit(Xtrain,Ytrain,batch_size=4, epochs=500,verbose=1,validation_data=(Xval,Yval),
        callbacks=[tensorboard_callback])

Epoch 1/500
17/17 [==============================] - 36s 2s/step - loss: 20.8943 - mse: 1257.8346 - val_loss: 24.8944 - val_mse: 1640.4677
Epoch 2/500
17/17 [==============================] - 23s 1s/step - loss: 20.8380 - mse: 1252.9521 - val_loss: 24.8912 - val_mse: 1640.4641
Epoch 3/500
17/17 [==============================] - 23s 1s/step - loss: 20.8388 - mse: 1252.9512 - val_loss: 24.8910 - val_mse: 1640.4639
Epoch 4/500
17/17 [==============================] - 23s 1s/step - loss: 20.8377 - mse: 1252.9512 - val_loss: 24.8959 - val_mse: 1640.4648
Epoch 5/500
17/17 [==============================] - 23s 1s/step - loss: 20.8384 - mse: 1252.9510 - val_loss: 24.8906 - val_mse: 1640.4641
Epoch 6/500
17/17 [==============================] - 23s 1s/step - loss: 20.8378 - mse: 1252.9512 - val_loss: 24.8869 - val_mse: 1640.4634
Epoch 7/500
17/17 [==============================] - 23s 1s/step - loss: 20.8385 - mse: 1252.9512 - val_loss: 24.8976 - val_mse: 1640.4656
Epoch 8/500
17/17 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [50]:
Xtrain.shape

TensorShape([65, 12, 300, 300, 3])

In [53]:
seq.predict(Xtest).shape

(24, 12, 300, 300, 3)

In [54]:
Ytest.shape

TensorShape([24, 12, 300, 300, 3])

In [58]:
pred=seq.predict(Xtest)

In [59]:
teue=Ytest

In [61]:
pred_=pred[:,:,:,:,1].flatten()

In [63]:
true_=np.array(teue[:,:,:,:,1]).flatten()

In [67]:
np.sum(np.abs(pred_-true_))/len(true_)

0.48878364197530866